# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

In [1]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process labeled data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [3]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

In [4]:
# contorna erro legacy
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [5]:
import pandas as pd
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [6]:
import torch
torch.cuda.is_available()

True

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [7]:
#!mkdir -p /content/drive/MyDrive/IC/unirTeste
#!wget -qnc -P /content/drive/MyDrive/IC/unirTeste/ http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured.zip

In [8]:
#!unzip -q /content/drive/MyDrive/IC/unirTeste/Structured.zip -d /content/drive/MyDrive/IC/unirTeste

In [9]:
#!ls /content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar

In [10]:
'''
def merge_data(labeled, table_a, table_b, output):
  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})
  table_a_csv = pd.read_csv(table_a)
  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})
  table_b_csv = pd.read_csv(table_b)
  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})
  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')
  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')
  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)
  del merged_csv['left_id']
  del merged_csv['right_id']
  merged_csv.to_csv(output, index=False)
'''

"\ndef merge_data(labeled, table_a, table_b, output):\n  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})\n  table_a_csv = pd.read_csv(table_a)\n  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})\n  table_b_csv = pd.read_csv(table_b)\n  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})\n  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')\n  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')\n  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)\n  del merged_csv['left_id']\n  del merged_csv['right_id']\n  merged_csv.to_csv(output, index=False)\n"

In [11]:
'''
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_test.csv')
'''

"\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/

To get an idea of how our data looks like, let's take a peek at the training dataset:

In [12]:
#pd.read_csv('/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Structured/DBLP-GoogleScholar/joined_train.csv').head()

## Step 1. Process labeled data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/data_processing.ipynb)**. 

### Processing labeled data
In order to process our train, validation and test CSV files we call `dm.data.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.data.process` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.data.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [11]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip --directory-prefix=/root/.vector_cache
!unzip /root/.vector_cache/wiki.en.zip -d /root/.vector_cache/
!rm /root/.vector_cache/wiki.en.vec

--2022-08-09 20:56:29--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’

wiki.en.zip          91%[=================>  ]   8.82G  29.4MB/s    in 3m 35s  

2022-08-09 21:00:05 (42.0 MB/s) - Read error at byte 9466635303/10356881291 (Connection reset by peer). Retrying.

--2022-08-09 21:00:06--  (try: 2)  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 10356881291 (9.6G), 890245988 (849M) remaining [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’



In [12]:
'''train, validation, test = dm.data.process(
    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test_error.csv')'''

"train, validation, test = dm.data.process(\n    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',\n    train='joined_train.csv',\n    validation='joined_valid.csv',\n    test='joined_test_error.csv')"

#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [13]:
'''train_table = train.get_raw_table()
train_table.head()'''

'train_table = train.get_raw_table()\ntrain_table.head()'

The processed attribute values have been tokenized and lowercased so they may not look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match. This is the default.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/matching_models.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [14]:
'''model = dm.MatchingModel(attr_summarizer='attention')'''

"model = dm.MatchingModel(attr_summarizer='attention')"

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_neg_ratio**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [15]:
'''model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)'''

"model.run_train(\n    train,\n    validation,\n    epochs=15,\n    batch_size=32,\n    best_save_path='attention_model.pth',\n    pos_neg_ratio=3)"

## Novos Testes

In [11]:
!git clone https://github.com/pauloh48/IC.git

fatal: destination path 'IC' already exists and is not an empty directory.


### RNN

#### STRUCTURED

##### Amazon-Google

In [18]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [19]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [20]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    5.9 | Load Time:    4.9 || F1:  17.54 | Prec:  37.56 | Rec:  11.44 || Ex/s: 636.37

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    5.5 | Load Time:    4.9 || F1:  99.36 | Prec:  99.01 | Rec:  99.71 || Ex/s: 666.13

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    1.5 || F1:  39.80 | Prec:  47.62 | Rec:  34.19 || Ex/s: 962.15

---------------------

Loading best model...
Training done.


tensor(44.5833, device='cuda:0')

In [21]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 3 || Run Time:    0.8 | Load Time:    1.5 || F1:  41.08 | Prec:  39.92 | Rec:  42.31 || Ex/s: 961.33



tensor(41.0788, device='cuda:0')

##### Beer

In [22]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [23]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [24]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 418.57

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 568.93

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.3 | Load Time:    0.3 || F1:  26.09 | Prec: 100.00 | Rec:  15.00 || Ex/s: 434.87

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:  13.33 | Prec: 100.00 | Rec:   7.14 || Ex/s: 567.43

* Best F1: tensor(13.3333, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 433.05

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  50.00 | Prec:  35.29 | Rec:  85.71 || Ex/s: 624.35

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 437.04

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  50.00 | Prec:  35.29 | Rec:  85.71 || Ex/s: 621.71

---------------------

Loading best model...
Training done.


tensor(58.0645, device='cuda:0')

In [25]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 4 || Run Time:    0.0 | Load Time:    0.1 || F1:  35.29 | Prec:  30.00 | Rec:  42.86 || Ex/s: 610.95



tensor(35.2941, device='cuda:0')

##### DBLP-ACM

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 1 || Run Time:    9.5 | Load Time:   12.9 || F1:  89.87 | Prec:  84.01 | Rec:  96.62 || Ex/s: 330.97

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:    8.9 | Load Time:   12.3 || F1:  99.92 | Prec:  99.85 | Rec: 100.00 || Ex/s: 351.37

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    1.3 | Load Time:    3.8 || F1:  97.78 | Prec:  96.29 | Rec:  99.32 || Ex/s: 479.57

---------------------

Loading best model...
Training done.


tensor(98.4270, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 2 || Run Time:    1.3 | Load Time:    3.8 || F1:  97.10 | Prec:  96.24 | Rec:  97.97 || Ex/s: 478.70



tensor(97.0982, device='cuda:0')

##### DBLP-GoogleScholar

In [12]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [13]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [14]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:43


Finished Epoch 1 || Run Time:   19.2 | Load Time:   25.1 || F1:  77.41 | Prec:  71.10 | Rec:  84.94 || Ex/s: 388.19

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Finished Epoch 15 || Run Time:   19.8 | Load Time:   25.4 || F1:  99.94 | Prec:  99.97 | Rec:  99.91 || Ex/s: 380.76

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.0 | Load Time:    7.8 || F1:  90.21 | Prec:  89.59 | Rec:  90.84 || Ex/s: 534.73

---------------------

Loading best model...
Training done.


tensor(90.2663, device='cuda:0')

In [15]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 9 || Run Time:    3.0 | Load Time:    7.8 || F1:  89.57 | Prec:  88.08 | Rec:  91.12 || Ex/s: 533.35



tensor(89.5728, device='cuda:0')

##### Walmart-Amazon

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2576402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    7.9 | Load Time:    6.9 || F1:  26.27 | Prec:  57.65 | Rec:  17.01 || Ex/s: 415.86

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    8.0 | Load Time:    6.9 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 411.17

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    1.2 | Load Time:    2.2 || F1:  62.46 | Prec:  79.84 | Rec:  51.30 || Ex/s: 603.34

---------------------

Loading best model...
Training done.


tensor(62.8571, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 11 || Run Time:    1.4 | Load Time:    2.5 || F1:  64.80 | Prec:  81.25 | Rec:  53.89 || Ex/s: 523.96



tensor(64.7975, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 1 || Run Time:    6.7 | Load Time:   11.1 || F1:  20.31 | Prec:  31.72 | Rec:  14.94 || Ex/s: 322.99

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    6.7 | Load Time:   11.2 || F1:  99.68 | Prec:  99.35 | Rec: 100.00 || Ex/s: 320.43

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.9 | Load Time:    3.3 || F1:  25.89 | Prec:  38.83 | Rec:  19.42 || Ex/s: 452.06

---------------------

Loading best model...
Training done.


tensor(31.2821, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    1.0 | Load Time:    3.3 || F1:  30.85 | Prec:  34.12 | Rec:  28.16 || Ex/s: 447.36



tensor(30.8511, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25


Finished Epoch 1 || Run Time:   10.3 | Load Time:   15.6 || F1:  68.89 | Prec:  61.00 | Rec:  79.13 || Ex/s: 285.55

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 15 || Run Time:    9.8 | Load Time:   15.1 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 297.87

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    1.5 | Load Time:    4.7 || F1:  90.14 | Prec:  88.67 | Rec:  91.67 || Ex/s: 399.18

* Best F1: tensor(90.1440, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(90.1440, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 15 || Run Time:    1.5 | Load Time:    4.6 || F1:  90.40 | Prec:  89.60 | Rec:  91.22 || Ex/s: 405.61



tensor(90.4018, device='cuda:0')

##### DBLP-GoogleScholar

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [12]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:50


Finished Epoch 1 || Run Time:   20.6 | Load Time:   29.9 || F1:  64.28 | Prec:  57.30 | Rec:  73.21 || Ex/s: 341.04

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:50


Finished Epoch 15 || Run Time:   20.5 | Load Time:   30.4 || F1:  99.92 | Prec:  99.97 | Rec:  99.88 || Ex/s: 338.23

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 15 || Run Time:    3.0 | Load Time:    9.1 || F1:  82.48 | Prec:  82.25 | Rec:  82.71 || Ex/s: 472.37

---------------------

Loading best model...
Training done.


tensor(83.1514, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 11 || Run Time:    3.1 | Load Time:    9.2 || F1:  81.83 | Prec:  84.26 | Rec:  79.53 || Ex/s: 466.45



tensor(81.8269, device='cuda:0')

### SIF

#### STRUCTURED

##### Amazon-Google

In [11]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [12]:
model = dm.MatchingModel(attr_summarizer='sif')

In [13]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    3.7 | Load Time:    5.2 || F1:   4.22 | Prec:  16.04 | Rec:   2.43 || Ex/s: 766.73

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    2.9 | Load Time:    4.5 || F1:  94.74 | Prec:  91.70 | Rec:  98.00 || Ex/s: 928.39

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 15 || Run Time:    0.6 | Load Time:    1.4 || F1:  27.36 | Prec:  30.53 | Rec:  24.79 || Ex/s: 1136.17

---------------------

Loading best model...
Training done.


tensor(32.7068, device='cuda:0')

In [14]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Finished Epoch 5 || Run Time:    0.6 | Load Time:    1.4 || F1:  36.50 | Prec:  32.88 | Rec:  41.03 || Ex/s: 1119.99



tensor(36.5019, device='cuda:0')

##### Beer

In [15]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [16]:
model = dm.MatchingModel(attr_summarizer='sif')

In [17]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.3 || F1:  23.68 | Prec:  25.00 | Rec:  22.50 || Ex/s: 575.75

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 710.38

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 584.38

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 715.71

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.3 || F1:  63.33 | Prec:  95.00 | Rec:  47.50 || Ex/s: 612.75

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  40.00 | Prec:  66.67 | Rec:  28.57 || Ex/s: 699.31

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.3 || F1:  65.57 | Prec:  95.24 | Rec:  50.00 || Ex/s: 597.27

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  38.10 | Prec:  57.14 | Rec:  28.57 || Ex/s: 719.30

---------------------

Loading best model...
Training done.


tensor(40., device='cuda:0')

In [18]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Finished Epoch 9 || Run Time:    0.0 | Load Time:    0.1 || F1:  21.05 | Prec:  40.00 | Rec:  14.29 || Ex/s: 686.27



tensor(21.0526, device='cuda:0')

##### DBLP-ACM

In [13]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [14]:
model = dm.MatchingModel(attr_summarizer='sif')

In [15]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:    4.1 | Load Time:   11.9 || F1:  72.47 | Prec:  70.11 | Rec:  75.00 || Ex/s: 465.40

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:    3.9 | Load Time:   11.3 || F1:  99.81 | Prec:  99.63 | Rec: 100.00 || Ex/s: 489.29

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    3.5 || F1:  94.97 | Prec:  92.34 | Rec:  97.75 || Ex/s: 565.72

---------------------

Loading best model...
Training done.


tensor(95.0712, device='cuda:0')

In [16]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 13 || Run Time:    0.9 | Load Time:    3.8 || F1:  93.94 | Prec:  92.01 | Rec:  95.95 || Ex/s: 521.11



tensor(93.9360, device='cuda:0')

##### DBLP-GoogleScholar

In [17]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [18]:
model = dm.MatchingModel(attr_summarizer='sif')

In [19]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:    9.4 | Load Time:   24.8 || F1:  57.46 | Prec:  52.85 | Rec:  62.96 || Ex/s: 502.64

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    1.9 | Load Time:    7.2 || F1:  70.98 | Prec:  63.21 | Rec:  80.93 || Ex/s: 633.13

* Best F1: tensor(70.9836, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Finished Epoch 15 || Run Time:    8.8 | Load Time:   23.3 || F1:  99.56 | Prec:  99.63 | Rec:  99.50 || Ex/s: 536.32

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    1.9 | Load Time:    7.2 || F1:  78.08 | Prec:  79.29 | Rec:  76.92 || Ex/s: 630.99

---------------------

Loading best model...
Training done.


tensor(78.6517, device='cuda:0')

In [20]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 9 || Run Time:    1.9 | Load Time:    7.3 || F1:  77.26 | Prec:  79.92 | Rec:  74.77 || Ex/s: 624.48



tensor(77.2574, device='cuda:0')

##### Walmart-Amazon

In [21]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [22]:
model = dm.MatchingModel(attr_summarizer='sif')

In [23]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    4.1 | Load Time:    6.8 || F1:  18.03 | Prec:  51.22 | Rec:  10.94 || Ex/s: 564.63

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    0.8 | Load Time:    2.0 || F1:  44.02 | Prec:  40.89 | Rec:  47.67 || Ex/s: 721.32

* Best F1: tensor(44.0191, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.8 | Load Time:    6.4 || F1:  99.04 | Prec:  99.13 | Rec:  98.96 || Ex/s: 599.33

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    2.0 || F1:  51.65 | Prec:  61.43 | Rec:  44.56 || Ex/s: 716.56

---------------------

Loading best model...
Training done.


tensor(52.1212, device='cuda:0')

In [24]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 12 || Run Time:    0.8 | Load Time:    2.0 || F1:  55.90 | Prec:  69.77 | Rec:  46.63 || Ex/s: 710.30



tensor(55.9006, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [25]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [26]:
model = dm.MatchingModel(attr_summarizer='sif')

In [27]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:    2.7 | Load Time:   10.4 || F1:   0.32 | Prec:   9.09 | Rec:   0.16 || Ex/s: 440.35

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    0.5 | Load Time:    3.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 536.85

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 15 || Run Time:    2.5 | Load Time:   10.2 || F1:  97.07 | Prec:  94.88 | Rec:  99.35 || Ex/s: 452.22

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    0.5 | Load Time:    3.0 || F1:  19.19 | Prec:  23.91 | Rec:  16.02 || Ex/s: 539.08

---------------------

Loading best model...
Training done.


tensor(24.3902, device='cuda:0')

In [28]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 4 || Run Time:    0.5 | Load Time:    3.0 || F1:  19.92 | Prec:  17.13 | Rec:  23.79 || Ex/s: 539.15



tensor(19.9187, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [29]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [30]:
model = dm.MatchingModel(attr_summarizer='sif')

In [31]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 1 || Run Time:    3.9 | Load Time:   13.8 || F1:  38.03 | Prec:  52.30 | Rec:  29.88 || Ex/s: 418.07

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    0.8 | Load Time:    4.3 || F1:  61.41 | Prec:  57.59 | Rec:  65.77 || Ex/s: 481.76

* Best F1: tensor(61.4090, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    3.9 | Load Time:   13.8 || F1:  98.51 | Prec:  97.71 | Rec:  99.32 || Ex/s: 420.45

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    4.3 || F1:  66.90 | Prec:  68.56 | Rec:  65.32 || Ex/s: 484.46

---------------------

Loading best model...
Training done.


tensor(67.0927, device='cuda:0')

In [32]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 9 || Run Time:    0.8 | Load Time:    4.3 || F1:  70.53 | Prec:  66.21 | Rec:  75.45 || Ex/s: 486.86



tensor(70.5263, device='cuda:0')

##### DBLP-GoogleScholar

In [33]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [34]:
model = dm.MatchingModel(attr_summarizer='sif')

In [35]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Finished Epoch 1 || Run Time:    8.9 | Load Time:   27.9 || F1:  45.19 | Prec:  43.44 | Rec:  47.08 || Ex/s: 467.13

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    1.9 | Load Time:    8.5 || F1:  60.63 | Prec:  57.50 | Rec:  64.11 || Ex/s: 552.12

* Best F1: tensor(60.6275, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Finished Epoch 15 || Run Time:    8.8 | Load Time:   27.5 || F1:  99.03 | Prec:  99.22 | Rec:  98.85 || Ex/s: 473.53

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    1.9 | Load Time:    8.5 || F1:  66.63 | Prec:  69.47 | Rec:  64.02 || Ex/s: 553.17

---------------------

Loading best model...
Training done.


tensor(67.3738, device='cuda:0')

In [36]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 5 || Run Time:    1.9 | Load Time:    8.6 || F1:  65.92 | Prec:  61.42 | Rec:  71.12 || Ex/s: 549.03



tensor(65.9160, device='cuda:0')

### ATTENTION

#### STRUCTURED

##### Amazon-Google

In [37]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [38]:
model = dm.MatchingModel(attr_summarizer='attention')

In [39]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   11.3 | Load Time:    4.6 || F1:  24.19 | Prec:  34.11 | Rec:  18.74 || Ex/s: 433.07

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.1 | Load Time:    1.4 || F1:  37.22 | Prec:  28.81 | Rec:  52.56 || Ex/s: 648.21

* Best F1: tensor(37.2163, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:   11.2 | Load Time:    4.6 || F1:  92.54 | Prec:  90.22 | Rec:  94.99 || Ex/s: 434.26

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    2.1 | Load Time:    1.4 || F1:  42.49 | Prec:  46.23 | Rec:  39.32 || Ex/s: 658.16

---------------------

Loading best model...
Training done.


tensor(45.6835, device='cuda:0')

In [40]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    2.1 | Load Time:    1.4 || F1:  46.44 | Prec:  40.58 | Rec:  54.27 || Ex/s: 651.33



tensor(46.4351, device='cuda:0')

##### Beer

In [41]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [42]:
model = dm.MatchingModel(attr_summarizer='attention')

In [43]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.6 | Load Time:    0.3 || F1:   8.33 | Prec:   9.38 | Rec:   7.50 || Ex/s: 295.04

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:  27.03 | Prec:  21.74 | Rec:  35.71 || Ex/s: 418.30

* Best F1: tensor(27.0270, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.6 | Load Time:    0.3 || F1:  52.27 | Prec:  47.92 | Rec:  57.50 || Ex/s: 301.04

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 434.82

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.6 | Load Time:    0.3 || F1:  86.67 | Prec:  78.00 | Rec:  97.50 || Ex/s: 306.26

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.1 | Load Time:    0.1 || F1:  63.16 | Prec:  50.00 | Rec:  85.71 || Ex/s: 428.85

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.6 | Load Time:    0.3 || F1:  87.91 | Prec:  78.43 | Rec: 100.00 || Ex/s: 304.39

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.1 | Load Time:    0.1 || F1:  63.16 | Prec:  50.00 | Rec:  85.71 || Ex/s: 444.35

---------------------

Loading best model...
Training done.


tensor(63.1579, device='cuda:0')

In [44]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:    0.1 | Load Time:    0.1 || F1:  54.05 | Prec:  43.48 | Rec:  71.43 || Ex/s: 421.74



tensor(54.0541, device='cuda:0')

##### DBLP-ACM

In [45]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [46]:
model = dm.MatchingModel(attr_summarizer='attention')

In [47]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   16.1 | Load Time:   11.3 || F1:  82.08 | Prec:  74.21 | Rec:  91.82 || Ex/s: 270.40

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    3.1 | Load Time:    3.6 || F1:  91.36 | Prec:  84.09 | Rec: 100.00 || Ex/s: 370.37

* Best F1: tensor(91.3580, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26


Finished Epoch 15 || Run Time:   15.9 | Load Time:   11.2 || F1:  99.66 | Prec:  99.48 | Rec:  99.85 || Ex/s: 272.99

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.1 | Load Time:    3.6 || F1:  97.55 | Prec:  96.48 | Rec:  98.65 || Ex/s: 369.93

---------------------

Loading best model...
Training done.


tensor(98.1132, device='cuda:0')

In [48]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    3.1 | Load Time:    3.5 || F1:  97.23 | Prec:  95.84 | Rec:  98.65 || Ex/s: 370.58



tensor(97.2253, device='cuda:0')

##### DBLP-GoogleScholar

In [49]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [50]:
model = dm.MatchingModel(attr_summarizer='attention')

In [51]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   36.2 | Load Time:   23.4 || F1:  76.96 | Prec:  69.18 | Rec:  86.72 || Ex/s: 288.81

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 1 || Run Time:    7.0 | Load Time:    7.2 || F1:  86.71 | Prec:  82.70 | Rec:  91.12 || Ex/s: 404.85

* Best F1: tensor(86.7052, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:59


Finished Epoch 15 || Run Time:   36.2 | Load Time:   23.4 || F1:  99.45 | Prec:  99.41 | Rec:  99.50 || Ex/s: 289.08

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    7.0 | Load Time:    7.2 || F1:  88.37 | Prec:  90.18 | Rec:  86.64 || Ex/s: 404.41

---------------------

Loading best model...
Training done.


tensor(89.2645, device='cuda:0')

In [52]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    7.1 | Load Time:    7.2 || F1:  89.15 | Prec:  87.01 | Rec:  91.40 || Ex/s: 399.95



tensor(89.1522, device='cuda:0')

##### Walmart-Amazon

In [53]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [54]:
model = dm.MatchingModel(attr_summarizer='attention')

In [55]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 5234402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   15.6 | Load Time:    6.5 || F1:   8.45 | Prec:  22.39 | Rec:   5.21 || Ex/s: 278.68

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    3.0 | Load Time:    2.0 || F1:  23.48 | Prec:  72.97 | Rec:  13.99 || Ex/s: 405.68

* Best F1: tensor(23.4783, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Finished Epoch 15 || Run Time:   15.5 | Load Time:    6.4 || F1:  96.46 | Prec:  95.88 | Rec:  97.05 || Ex/s: 280.94

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.0 | Load Time:    2.0 || F1:  48.83 | Prec:  48.96 | Rec:  48.70 || Ex/s: 405.35

---------------------

Loading best model...
Training done.


tensor(55.1020, device='cuda:0')

In [56]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 3 || Run Time:    3.1 | Load Time:    2.0 || F1:  54.70 | Prec:  58.58 | Rec:  51.30 || Ex/s: 400.13



tensor(54.6961, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [57]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [58]:
model = dm.MatchingModel(attr_summarizer='attention')

In [59]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   10.8 | Load Time:   10.1 || F1:  15.68 | Prec:  22.56 | Rec:  12.01 || Ex/s: 275.01

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    2.0 | Load Time:    3.0 || F1:  30.39 | Prec:  28.51 | Rec:  32.52 || Ex/s: 384.59

* Best F1: tensor(30.3855, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   10.7 | Load Time:   10.0 || F1:  79.31 | Prec:  73.91 | Rec:  85.55 || Ex/s: 277.12

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    2.0 | Load Time:    3.0 || F1:  28.99 | Prec:  28.85 | Rec:  29.13 || Ex/s: 383.12

---------------------

Loading best model...
Training done.


tensor(35.0554, device='cuda:0')

In [60]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    2.1 | Load Time:    3.0 || F1:  32.80 | Prec:  25.92 | Rec:  44.66 || Ex/s: 377.84



tensor(32.7986, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [61]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [62]:
model = dm.MatchingModel(attr_summarizer='attention')

In [63]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   16.4 | Load Time:   13.5 || F1:  59.02 | Prec:  51.07 | Rec:  69.89 || Ex/s: 247.38

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.2 | Load Time:    4.3 || F1:  80.51 | Prec:  76.73 | Rec:  84.68 || Ex/s: 329.43

* Best F1: tensor(80.5139, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:29


Finished Epoch 15 || Run Time:   16.5 | Load Time:   13.6 || F1:  97.22 | Prec:  96.18 | Rec:  98.27 || Ex/s: 246.91

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    3.2 | Load Time:    4.3 || F1:  86.86 | Prec:  88.17 | Rec:  85.59 || Ex/s: 328.37

---------------------

Loading best model...
Training done.


tensor(87.0455, device='cuda:0')

In [64]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 14


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 14 || Run Time:    3.3 | Load Time:    4.3 || F1:  90.26 | Prec:  89.76 | Rec:  90.77 || Ex/s: 326.84



tensor(90.2576, device='cuda:0')

##### DBLP-GoogleScholar

In [65]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [66]:
model = dm.MatchingModel(attr_summarizer='attention')

In [67]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   38.9 | Load Time:   28.4 || F1:  66.22 | Prec:  57.99 | Rec:  77.17 || Ex/s: 255.70

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:    7.2 | Load Time:    8.5 || F1:  76.25 | Prec:  67.14 | Rec:  88.22 || Ex/s: 366.82

* Best F1: tensor(76.2520, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:04


Finished Epoch 15 || Run Time:   37.8 | Load Time:   27.5 || F1:  94.96 | Prec:  92.75 | Rec:  97.29 || Ex/s: 263.57

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 15 || Run Time:    7.3 | Load Time:    8.5 || F1:  81.58 | Prec:  79.28 | Rec:  84.02 || Ex/s: 361.50

---------------------

Loading best model...
Training done.


tensor(81.6628, device='cuda:0')

In [68]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 2 || Run Time:    7.6 | Load Time:    8.8 || F1:  81.79 | Prec:  81.72 | Rec:  81.87 || Ex/s: 349.61



tensor(81.7927, device='cuda:0')

### HYBRID

#### STRUCTURED

##### Amazon-Google

In [69]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [70]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [71]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   18.7 | Load Time:    4.9 || F1:  23.18 | Prec:  32.47 | Rec:  18.03 || Ex/s: 291.03

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    3.2 | Load Time:    1.5 || F1:  36.21 | Prec:  29.97 | Rec:  45.73 || Ex/s: 489.04

* Best F1: tensor(36.2098, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 15 || Run Time:   18.6 | Load Time:    4.8 || F1:  98.30 | Prec:  97.60 | Rec:  99.00 || Ex/s: 294.14

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    3.2 | Load Time:    1.5 || F1:  43.93 | Prec:  48.45 | Rec:  40.17 || Ex/s: 483.77

---------------------

Loading best model...
Training done.


tensor(49.7925, device='cuda:0')

In [72]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    3.2 | Load Time:    1.5 || F1:  48.16 | Prec:  46.09 | Rec:  50.43 || Ex/s: 482.85



tensor(48.1633, device='cuda:0')

##### Beer

In [73]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [74]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [75]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    1.1 | Load Time:    0.3 || F1:   8.33 | Prec:   9.38 | Rec:   7.50 || Ex/s: 194.89

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 328.24

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    1.0 | Load Time:    0.3 || F1:  48.15 | Prec:  92.86 | Rec:  32.50 || Ex/s: 199.94

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 340.50

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    1.0 | Load Time:    0.3 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 202.17

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.1 || F1:  78.79 | Prec:  68.42 | Rec:  92.86 || Ex/s: 339.45

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    1.0 | Load Time:    0.3 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 202.03

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.1 || F1:  78.79 | Prec:  68.42 | Rec:  92.86 || Ex/s: 330.66

---------------------

Loading best model...
Training done.


tensor(81.2500, device='cuda:0')

In [76]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:    0.2 | Load Time:    0.1 || F1:  64.52 | Prec:  58.82 | Rec:  71.43 || Ex/s: 337.48



tensor(64.5161, device='cuda:0')

##### DBLP-ACM

In [77]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [78]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [79]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   28.3 | Load Time:   11.7 || F1:  84.49 | Prec:  77.09 | Rec:  93.47 || Ex/s: 185.28

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    4.9 | Load Time:    3.8 || F1:  94.74 | Prec:  90.55 | Rec:  99.32 || Ex/s: 284.46

* Best F1: tensor(94.7368, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


Finished Epoch 15 || Run Time:   28.2 | Load Time:   11.7 || F1:  99.89 | Prec:  99.78 | Rec: 100.00 || Ex/s: 185.86

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    4.9 | Load Time:    3.8 || F1:  97.74 | Prec:  97.96 | Rec:  97.52 || Ex/s: 284.20

---------------------

Loading best model...
Training done.


tensor(98.3203, device='cuda:0')

In [80]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 4 || Run Time:    5.1 | Load Time:    3.8 || F1:  97.99 | Prec:  97.12 | Rec:  98.87 || Ex/s: 277.81



tensor(97.9911, device='cuda:0')

##### DBLP-GoogleScholar

In [81]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [82]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [83]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   63.6 | Load Time:   23.9 || F1:  79.09 | Prec:  71.70 | Rec:  88.18 || Ex/s: 196.74

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


Finished Epoch 1 || Run Time:   11.1 | Load Time:    7.8 || F1:  89.60 | Prec:  89.40 | Rec:  89.81 || Ex/s: 304.57

* Best F1: tensor(89.6037, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:25


Finished Epoch 15 || Run Time:   62.8 | Load Time:   23.6 || F1:  99.74 | Prec:  99.60 | Rec:  99.88 || Ex/s: 199.30

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Finished Epoch 15 || Run Time:   11.6 | Load Time:    7.9 || F1:  91.87 | Prec:  92.93 | Rec:  90.84 || Ex/s: 294.37

---------------------

Loading best model...
Training done.


tensor(92.2719, device='cuda:0')

In [84]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:   11.3 | Load Time:    7.9 || F1:  91.28 | Prec:  91.07 | Rec:  91.50 || Ex/s: 299.41



tensor(91.2821, device='cuda:0')

##### Walmart-Amazon

In [85]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [86]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [87]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 1 || Run Time:   26.8 | Load Time:    6.8 || F1:  34.01 | Prec:  53.96 | Rec:  24.83 || Ex/s: 183.20

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Finished Epoch 15 || Run Time:   26.6 | Load Time:    6.7 || F1:  99.31 | Prec:  99.13 | Rec:  99.48 || Ex/s: 184.34

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    4.7 | Load Time:    2.1 || F1:  60.22 | Prec:  62.57 | Rec:  58.03 || Ex/s: 301.36

---------------------

Loading best model...
Training done.


tensor(62.0321, device='cuda:0')

In [88]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    4.7 | Load Time:    2.2 || F1:  59.57 | Prec:  61.20 | Rec:  58.03 || Ex/s: 296.00



tensor(59.5745, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [89]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [90]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [91]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   18.1 | Load Time:   10.3 || F1:  15.73 | Prec:  25.55 | Rec:  11.36 || Ex/s: 201.93

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 1 || Run Time:    3.1 | Load Time:    3.2 || F1:  33.72 | Prec:  25.70 | Rec:  49.03 || Ex/s: 305.32

* Best F1: tensor(33.7229, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Finished Epoch 15 || Run Time:   18.0 | Load Time:   10.3 || F1:  97.99 | Prec:  97.28 | Rec:  98.70 || Ex/s: 203.27

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.1 | Load Time:    3.2 || F1:  54.64 | Prec:  58.24 | Rec:  51.46 || Ex/s: 305.55

---------------------

Loading best model...
Training done.


tensor(54.7739, device='cuda:0')

In [92]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 8 || Run Time:    3.4 | Load Time:    3.3 || F1:  53.20 | Prec:  54.00 | Rec:  52.43 || Ex/s: 285.82



tensor(53.2020, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [93]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [94]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [95]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   29.1 | Load Time:   14.1 || F1:  59.98 | Prec:  50.94 | Rec:  72.90 || Ex/s: 171.31

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    5.2 | Load Time:    4.7 || F1:  69.56 | Prec:  54.63 | Rec:  95.72 || Ex/s: 251.11

* Best F1: tensor(69.5581, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Finished Epoch 15 || Run Time:   28.9 | Load Time:   13.9 || F1:  99.63 | Prec:  99.40 | Rec:  99.85 || Ex/s: 173.28

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    5.1 | Load Time:    4.6 || F1:  90.01 | Prec:  89.71 | Rec:  90.32 || Ex/s: 252.71

---------------------

Loading best model...
Training done.


tensor(90.9900, device='cuda:0')

In [96]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 13 || Run Time:    5.4 | Load Time:    4.6 || F1:  89.29 | Prec:  89.39 | Rec:  89.19 || Ex/s: 247.23



tensor(89.2897, device='cuda:0')

##### DBLP-GoogleScholar

In [97]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/BDirty_50_3_5/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [98]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [99]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   64.7 | Load Time:   28.0 || F1:  70.53 | Prec:  62.50 | Rec:  80.92 || Ex/s: 185.85

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 1 || Run Time:   11.7 | Load Time:    9.1 || F1:  83.70 | Prec:  78.43 | Rec:  89.72 || Ex/s: 275.60

* Best F1: tensor(83.6966, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:32


Finished Epoch 15 || Run Time:   65.2 | Load Time:   27.8 || F1:  99.72 | Prec:  99.69 | Rec:  99.75 || Ex/s: 185.11

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   11.6 | Load Time:    9.1 || F1:  88.50 | Prec:  88.50 | Rec:  88.50 || Ex/s: 276.33

---------------------

Loading best model...
Training done.


tensor(89.3736, device='cuda:0')

In [100]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:   12.0 | Load Time:    9.3 || F1:  90.38 | Prec:  89.03 | Rec:  91.78 || Ex/s: 269.83



tensor(90.3820, device='cuda:0')

## Step 4. Apply model to new data

### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [ ]:
# Compute F1 on test set
'''model.run_eval(test)'''

### Evaluating on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we need to first process the unlabeled data.

#### Processing unlabeled data

To process unlabeled data, we use `dm.data.process_unlabeled`, as shown in the code snippet below. The basic parameters for this call are as follows:

* **path (required): ** The full path to the unlabeled data file (not just the directory).
* **trained_model (required): ** The trained model. The model is aware of the configuration of the training data on which it was trained, and so `deepmatcher` reuses the same configuration for the unlabeled data.
* **ignore_columns (optional): ** Any columns in the unlabeled CSV file that you may want to ignore for the purposes of evaluation. If not specified, the columns that were ignored while processing the training set will also be ignored while processing the unlabeled data.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [ ]:
'''
unlabeled = dm.data.process_unlabeled(
    path='sample_data/itunes-amazon/unlabeled.csv',
    trained_model=model)
'''

#### Obtaining predictions

Next, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match score predictions (`match_score` column). `match_scores` are in [0, 1] and a score above 0.5 indicates a match prediction.

In [ ]:
'''
predictions = model.run_prediction(unlabeled)
predictions.head()
'''

You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [ ]:
#predictions = model.run_prediction(unlabeled, output_attributes=True)
#predictions.head()

You can then save these predictions to CSV and use them for downstream tasks.

In [ ]:
#predictions.to_csv('sample_data/itunes-amazon/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [ ]:
#valid_predictions = model.run_prediction(validation, output_attributes=True)
#valid_predictions.head()